# Evaluating BridgeStan Speed

This notebook is meant to evaluate the speed of BridgeStan relative to [issue #190](https://github.com/roualdes/bridgestan/issues/190). The goal is to offer users direct access Stan's methods via both numpy arrays and ctypes.  The question is which method is fastest and which method will be reasonable to maintain into the future.  For speed comparisons, BridgeStan's `log_density_gradient(...)` will serve as the baseline.

There are two strategies (to offering APIs for numpy arrays and ctypes) being evaluated.  The function `log_density_gradient_proposed(...)` uses `array_ptr` to differentiate between numpy arrays and ctypes `double*`s with an implicit if statement. An alternative solution is to expose two functions, one specifically for ctypes and one for numpy arrays.  The function `log_density_gradient_ctypes(...)` offers direct access to ctypes, while `log_density_gradient_alternative(...)` deals with numpy arrays, but internally just calls `log_density_gradient_ctypes(...)`.

### Clone and Install BridgeStan

In [1]:
# start fresh?
!rm -rf ./bridgestan

In [2]:
!git clone --recurse-submodules --shallow-submodules --depth=1 https://github.com/roualdes/bridgestan.git
!cd ./bridgestan && git fetch origin python/enable-ctypes-double-pointers:python/enable-ctypes-double-pointers && git checkout python/enable-ctypes-double-pointers

Cloning into 'bridgestan'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 184 (delta 3), reused 109 (delta 1), pack-reused 0
Receiving objects: 100% (184/184), 145.52 KiB | 1.05 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Submodule 'stan' (https://github.com/stan-dev/stan) registered for path 'stan'
Cloning into '/Users/ez/bridgestan-speed/bridgestan/stan'...
remote: Enumerating objects: 620, done.        
remote: Counting objects: 100% (620/620), done.        
remote: Compressing objects: 100% (519/519), done.        
remote: Total 620 (delta 148), reused 296 (delta 77), pack-reused 0        
Receiving objects: 100% (620/620), 4.63 MiB | 4.28 MiB/s, done.
Resolving deltas: 100% (148/148), done.
remote: Total 0 (delta 0), reused 0 (delta 0), pack-reused 0
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100%

In [3]:
!cd ./bridgestan && pip install ./python

Processing ./python
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for bridgestan: filename=bridgestan-2.2.2-py3-none-any.whl size=11744 sha256=f583eca3959dcf60701498634a29ac236d87123487b68c302d90748401e9f534
  Stored in directory: /private/var/folders/10/vhgkp_1x0p310y0lw2d5mx8h0000gn/T/pip-ephem-wheel-cache-hw30l9bm/wheels/c0/af/43/4c10ee4e3df14332c19c8b6a53ff3b068ac02d4af336917a59
Successfully built bridgestan
  Attempting uninstall: bridgestan
    Found existing installation: bridgestan 2.2.2
    Uninstalling bridgestan-2.2.2:
      Successfully uninstalled bridgestan-2.2.2

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


### Clone and Install ExperimentalHMC (for testing direct ctypes access)

In [4]:
!git clone git@github.com:roualdes/experimentalHMC.git
!cd ./experimentalHMC && git fetch origin start && git checkout start
!cd ./experimentalHMC && pip install .

Cloning into 'experimentalHMC'...
remote: Enumerating objects: 799, done.
remote: Counting objects: 100% (799/799), done.
remote: Compressing objects: 100% (587/587), done.
remote: Total 799 (delta 307), reused 679 (delta 189), pack-reused 0
Receiving objects: 100% (799/799), 1.21 MiB | 2.65 MiB/s, done.
Resolving deltas: 100% (307/307), done.
From github.com:roualdes/experimentalHMC
 * branch            start      -> FETCH_HEAD
branch 'start' set up to track 'origin/start'.
Switched to a new branch 'start'
Processing /Users/ez/bridgestan-speed/experimentalHMC
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experimentalhmc: filename=experimentalhmc-0.0.1-cp311-cp311-macosx_12_0_x86_64.whl size=28055 sha256=c28558b59b0c6912d8f75fbb716b86c5c667685d04ec1c79dde8a8c78ff851c0
  Stored in directory: /private/var/folders/10/vhgkp_1x0p310y0lw2d5mx8h0000gn/T/pip-ephem-wheel-cache-6rd_6p2w/w

### Evaluate Numpy Access

In [5]:
import bridgestan as bs
import numpy as np

bs.set_bridgestan_path("./bridgestan")

In [6]:
model = "logistic"

m = bs.StanModel(f"test_models/{model}/{model}.stan",
                  data = f"test_models/{model}/{model}.data.json")

dims = m.param_unc_num()
R = 1_000
out = np.zeros(dims)

In [7]:
%%timeit -n 10
for r in range(R):
    q = np.random.normal(size = dims)
    m.log_density_gradient(q, out = out)

83.2 ms ± 4.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%%timeit -n 10
for r in range(R):
    q = np.random.normal(size = dims)
    m.log_density_gradient_proposed(q, out = out)

80.5 ms ± 1.43 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%%timeit -n 10
for r in range(R):
    q = np.random.normal(size = dims)
    m.log_density_gradient_alternative(q, out = out)

84.4 ms ± 338 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Evaluate Ctypes Access

In [10]:
import experimentalhmc as ehmc
from numpy.ctypeslib import as_array

In [11]:
def bridgestan_log_density_gradient_wrapper(bsm):
    dim = bsm.param_unc_num()
    def bsm_c_wrapper(position, gradient):
        ld, _ = bsm.log_density_gradient(as_array(position, shape = (dims,)), 
                                      out = as_array(gradient, shape = (dims,)))
        return ld
    return bsm_c_wrapper

def bridgestan_log_density_gradient_proposed_wrapper(bsm):
    dim = bsm.param_unc_num()
    def bsm_c_wrapper(position, gradient):
        ld, _ = bsm.log_density_gradient_proposed(position, out = gradient)
        return ld
    return bsm_c_wrapper
    
def bridgestan_log_density_gradient_ctypes_wrapper(bsm):
    dim = bsm.param_unc_num()
    def bsm_c_wrapper(position, gradient):
        ld = bsm.log_density_gradient_ctypes(position, gradient)
        return ld
    return bsm_c_wrapper

In [12]:
ldg = bridgestan_log_density_gradient_wrapper(m)
ldg_proposed = bridgestan_log_density_gradient_proposed_wrapper(m)
ldg_ctypes = bridgestan_log_density_gradient_ctypes_wrapper(m)

In [13]:
stan = ehmc.Stan(dims, ldg)
stan_proposed = ehmc.Stan(dims, ldg_proposed)
stan_ctypes = ehmc.Stan(dims, ldg_ctypes)

In [14]:
%%timeit -n 10
for m in range(1000):
    stan.sample()

2.41 s ± 222 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%%timeit -n 10
for m in range(1000):
    stan_proposed.sample()

1.95 s ± 99.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%%timeit -n 10
for m in range(1000):
    stan_ctypes.sample()

1.84 s ± 10.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
